#[문제]
- law_2.docx 파일을 읽고, chroma 저장  
- LLM질문 -> 답변  
- 전세사기피해에 관한 법률 질문만 받기.
- 이 외의 질문은 '답변을 할 수 없습니다.'

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader

In [3]:
## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 

## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


In [4]:
# # ## 2.3. 벡터 데이터베이스에 저장
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

## 로컬에 저장된 임베딩 데이터 가져오기

database =Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

database


In [6]:
## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색

## 3.1. 사용자 질문
query = '오늘 날씨는?'
# query = '전세사기 피해자 지원 방법은? '

## 3.2. 벡터 데이터베이스에서 유사도 검색 (점수 포함)
retrieved_docs_with_score = database.similarity_search_with_score(query=query, k=3)

## 3.3. 점수 필터링 (예: 0.8 이상만 유효한 문서로 간주)

threshold = 0.8
filtered_docs = [doc for doc, score in retrieved_docs_with_score if score >= threshold]

## 4.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

## 4.1. 프롬프트 작성
if not filtered_docs:
    print('답변을 할 수 없습니다.')
    exit()
    
else:
    prompt = '''
    [identity]
    - 당신은 전세사기피해 법률 전문가 입니다.
    - [context]를 참고하여 사용자의 질문에 답변해주세요.

    [context]
    {retrieved_docs}

    Question: {query}
    '''
    ## 4.2 프롬프트 변수에 값 설정
    formatted_prompt = prompt.format(
        retrieved_docs = "\n\n".join([doc.page_content for doc in filtered_docs]),
        query=query,)

## 4.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='죄송하지만, 전세사기피해와 관련된 질문이라면 도와드릴 수 있습니다. 전세사기피해와 관련하여 궁금한 점이 있으시다면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 2955, 'total_tokens': 2999, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2944}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-Bdwhcw05V1Myjt5lxCebItQIhMYMB', 'finish_reason': 'stop', 'logprobs': None}, id='run--1e7f40d3-8889-416e-a309-fe19c006a7ac-0', usage_metadata={'input_tokens': 2955, 'output_tokens': 44, 'total_tokens': 2999, 'input_token_details': {'audio': 0, 'cache_read': 2944}, 'output_token_details': {'audio': 0, 'reasoning': 0}})